In [11]:
!unzip  /home/aistudio/data/data102252/train_50k_mask.zip -d data/
!unzip  /home/aistudio/data/data102252/train_image.zip -d data/
!unzip  /home/aistudio/data/data102252/Btest.zip  -d data/

比赛背景
第三届中国AI+创新创业大赛由中国人工智能学会主办，半监督学习目标定位竞赛分赛道要求选手基于少量有标注数据训练模型，使分类网络具有目标定位能力，实现半监督目标定位任务。

中国人工智能学会（Chinese Association for Artificial Intelligence，CAAI）成立于1981年，是经国家民政部正式注册的我国智能科学技术领域唯一的国家级学会，是全国性4A级社会组织，挂靠单位为北京邮电大学；是中国科学技术协会的正式团体会员，具有推荐“两院院士”的资格。

中国人工智能学会目前拥有51个分支机构，包括43个专业委员会和8个工作委员会，覆盖了智能科学与技术领域。学会活动的学术领域是智能科学技术，活动地域是中华人民共和国全境，基本任务是团结全国智能科学技术工作者和积极分子通过学术研究、国内外学术交流、科学普及、学术教育、科技会展、学术出版、人才推荐、学术评价、学术咨询、技术评审与奖励等活动促进我国智能科学技术的发展，为国家的经济发展、社会进步、文明提升、安全保障提供智能化的科学技术服务。

中国“AI+”创新创业大赛由中国人工智能学会发起主办，是为了配合实施创新驱动助力工程，深入开展服务企业技术创新活动，进一步提高我国文化建设和实践创新能力，展示智能科学与技术等相关学科建设的新经验、新成果，促进专业内涵的建设而发起的综合性大赛平台。

飞桨PaddlePaddle作为中国首个自主研发、功能完备、开源开放的产业级深度学习平台，为本次比赛的参赛选手提供了集深度学习核心训练和推理框架、基础模型库、端到端开发套件和丰富的工具组件于一体的一站式服务。百度大脑AI Studio作为官方指定且唯一的竞赛日常训练平台，为参赛选手提供高效的学习和开发环境，更有亿元Tesla V100算力免费赠送，助力选手取得优异成绩。


数据增强策略
```
1. transforms:
1.     - type: ResizeStepScaling
1.       min_scale_factor: 0.5
1.       max_scale_factor: 2
1.       scale_step_size: 0.25
1.     - type: RandomHorizontalFlip
1.     - type: RandomRotation1
1.     - type: RandomRotation2 
1.     - type: RandomBlur
1.     - type: RandomDistort
1.       brightness_range: 0.4
1.       brightness_prob: 1
1.       contrast_range: 0.4
1.       contrast_prob: 1
1.       saturation_range: 0.4
1.       saturation_prob: 1
1.       hue_range: 60
1.       hue_prob: 1
1.     - type: Resize
1.       target_size: [354,354]      
1.     - type: Normalize
1.       mean: [0.485, 0.456, 0.406]
1.       std: [0.229, 0.224, 0.225]
```
```
数据增强的过程中自定义了两种旋转策略，RandomRotation1和RandomRotation2改写了原有的transform.py中的关于RandomRotation的函数，
将其分别固定为顺时针旋转90 的RandomRotation1和逆时针旋转90的RandomRotation2,
为了提升效果将默认的RandomDistort的概率调整到1
为了使网络适应不同尺寸图片，采取ResizeStepScaling策略，将图片以0.25的步长从0.5倍扩展到2倍倍率
对于Normalize 使用Imagenet的均值和标准差
在一开始尝试 resize 图片尺寸较小 为[224,224]，后来将图片resize到[354,354]效果提升明显
后续尝试了更大的尺寸，但由于显存限制，只能将batchsize 调小，导致效果波动大，训练速度较慢
于是没有继续使用更大的尺寸
```
```
后面还尝试了使用Cutout HideAndSeek fmix GridMask
Coutout的思想可以参见论文，地址：https://arxiv.org/pdf/1708.04552.pdf
Cutout 可以理解为 Dropout 的一种扩展操作，不同的是 Dropout 是对图像经过网络后生成的特征进行遮挡，而 Cutout 是直接对输入的图像进行遮挡，相对于Dropout对噪声的鲁棒性更好。
hide_and_seek的思想可以参见论文，地址：https://arxiv.org/pdf/1811.02545.pdf
HideAndSeek将图像分为若干块区域(patch)，对于每块区域，都以一定的概率生成掩码
PaddleClas中已有该方法的实现
GridMask的思想可以参见论文，地址:https://arxiv.org/pdf/2001.04086.pdf
GridMask是先根据一系列超参数生成一个掩码图像，然后用该掩码图和原图相乘从而实现对原图进行有规则的信息舍弃的数据增强方法
FMix的思想可以参见论文，地址: https://arxiv.org/pdf/2002.12047.pdf
FMix是在综合Mixup和Cutmix的优点的基础上提出的算法，FMix的思路和Cutmix类似，只不过将固定的矩形ROI区域替换成了低频傅立叶抽样后的不同形状的ROI区域
但经过测试，未能产生明显效果
将数据集按照9:1划分 45000张训练，5000张验证
```
```
模型方案 
模型先后采取使用了DeepLabV3P,emanet,ocrnet dnlnet ,其与resnet和hrnet作为backbone组合
miou约能达到0.85到0.87左右
最终发现MLATransformer在这个问题效果更佳，miou能达到0.88
backbone 先后选取了hrnet,resnet,实验效果显示 swinTransform效果较好 ,
backbone的预训练模型选取paddlclas：https://github.com/PaddlePaddle/PaddleClas/blob/release/2.2/docs/en/ImageNet_models.md
调优过程，分阶段采取不同学习率，在初始阶段 采用较大学习率    
迭代约30000轮将这一阶段模型保存，在下一阶段以这个阶段为基础进行finetune，这一阶段采取更低的学习率，将学习率调整至0.001，迭代约80000轮 
在finetune阶段 分别采取不同的损失函数 如DiceLoss, BootstrappedCrossEntropyLoss， LovaszSoftmaxLoss，观察到效果提升不明显，有一定波动
后面进一步尝试将训练所遍历的文件进行逆序操作，使得模型从另一个顺序遍历文件，最终精度 miou约为 0.88左右
```
```

预测过程增强 采用多尺度策略，一开始尝试 尺度为 1.0 1.5 2.0，发现对于小目标识别效果不明显
而继续放大尺度后到2.5，发现其对小目标的识别有一定幅度提升

后续改进策略，加入其他模型进行融合
```

```
代码参考 Paddleseg库 https://github.com/PaddlePaddle/PaddleSeg/tree/release/2.2/paddleseg/models
https://github.com/PaddlePaddle/PaddleSeg/tree/release/2.2/paddleseg/models/backbones
以及 PaddleCla
https://github.com/PaddlePaddle/PaddleClas/tree/a6d927a122387642d04bb0ebb5785e1d7c74f78f/ppcls/data/preprocess/ops
```


In [15]:
import sys
import paddle
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm
import random


In [22]:

random.seed(1888)
def create(file_name, imgs_path, labels_path=None,val_pro=0.1):
        train_path = []
        for idx, f_path in enumerate(imgs_path):
            for i_path in sorted(os.listdir(f_path)):
                path1 = os.path.join(f_path, i_path) 
                path2 = os.path.join(labels_path[idx], i_path)
                train_path.append((path1, path2))
        
        if val_pro>=0 and val_pro<=1:
            random.shuffle(train_path)
            val_len = int(len(train_path)*val_pro)
            val_path = train_path[:val_len]
            train_path = train_path[val_len:]
            with open(file_name[0], 'w') as f:
                for path in train_path:
                    f.write(path[0]+" "+path[1]+"\n")
            with open(file_name[1], 'w') as f:
                for path in val_path:
                    f.write(path[0]+" "+path[1]+"\n")  
            return len(train_path), val_len
        else:
            with open(file_name[0], 'w') as f:
                for path in train_path:
                    f.write(path[0]+" "+path[1]+"\n") 
            return len(train_path), 0
    

In [24]:

cnt=0.1
roo = "data"
origin= "train_image"
mask = "train_50k_mask"

random.seed(1888)
origin= os.path.join(roo, origin)
mask = os.path.join(roo, mask)
file_names = os.listdir(origin)
file_names = sorted(file_names)
oripath, maskpath =[], []
for subs in file_names:
    oripath.append(os.path.join(origin, subs))
    maskpath.append(os.path.join(mask, subs))
    create(["train.txt", "val.txt"], oripath, maskpath, cnt) 

print("ok")

# 训练


In [ ]:
#base阶段
!python PaddleSeg/PaddleSeg/train.py --config base.yml --do_eval  --use_vdl --save_dir /home/aistudio/base// --save_interval 2000



In [28]:
#逆序train文件

with open('train.txt', 'r') as readf, open('train2.txt', 'w') as writef:
    for line in reversed(readf.readlines()):
        writef.write(line)


In [ ]:

#finetune阶段

!python PaddleSeg/PaddleSeg/train.py --config swin4.yml --do_eval  --use_vdl --save_dir /home/aistudio/swin4// --save_interval 1000

In [ ]:
!python PaddleSeg/PaddleSeg/train.py --config swin5.yml --do_eval  --use_vdl --save_dir /home/aistudio/swin5// --save_interval 1000

In [ ]:
!python PaddleSeg/PaddleSeg/train.py --config swin6.yml --do_eval  --use_vdl --save_dir /home/aistudio/swin6// --save_interval 1000

In [19]:
!python PaddleSeg/PaddleSeg/train.py --config swin8.yml --do_eval  --use_vdl --save_dir /home/aistudio/swin8// --save_interval 1000

In [29]:
#最佳模型日志
#查看最终精度的运行日志
from visualdl import LogReader

reader = LogReader(file_path='swin4/vdlrecords.1627531143.log')
tags = reader.get_tags()
print(tags)
data = reader.get_data('scalar', 'Evaluate/mIoU')
print(data)

{'meta_data': ['meta_data_tag'], 'scalar': ['Train/loss', 'Train/loss_0', 'Train/loss_1', 'Train/lr', 'Train/batch_cost', 'Train/reader_cost', 'Evaluate/mIoU', 'Evaluate/Acc']}
[id: 4000
tag: "Evaluate/mIoU"
timestamp: 1627533547811
value: 0.8789075613021851
, id: 8000
tag: "Evaluate/mIoU"
timestamp: 1627535928456
value: 0.882707953453064
, id: 12000
tag: "Evaluate/mIoU"
timestamp: 1627538324630
value: 0.8845129609107971
, id: 16000
tag: "Evaluate/mIoU"
timestamp: 1627540737411
value: 0.8848694562911987
, id: 20000
tag: "Evaluate/mIoU"
timestamp: 1627543143202
value: 0.8855800032615662
, id: 24000
tag: "Evaluate/mIoU"
timestamp: 1627545537942
value: 0.885741651058197
]


**#预测**

In [ ]:
#最佳结果保存在swin4文件夹
#第一种策略预测
!python PaddleSeg/PaddleSeg/predict.py --config swin4.yml --model_path swin4/best_model/model.pdparams --image_path data/test_image --save_dir /results --aug_pred --scales  1.0 1.5 2.0
 
 
 

In [ ]:

!python PaddleSeg/PaddleSeg/predict.py --config swin4.yml --model_path swin4/best_model/model.pdparams --image_path data/test_image --save_dir ans/results2 --aug_pred --scales 1.0 1.5 2.0 2.5
 
 
 